In [ ]:
!pip install -q segmentation-models-pytorch
!git clone -q https://github.com/mszczesniak02/bachlor_backup.git

In [ ]:
!gdown --id 1e8iUa45OsATWGrFJOVfE3x9-YZrWTQXG -O dataset.tar.gz
!mkdir /content/datasets
!tar -I pigz -xf /content/dataset.tar.gz -C /content/datasets/


In [ ]:
import os
import sys
from google.colab import drive
import glob
from pathlib import Path
import shutil
from tqdm.auto import tqdm

In [ ]:
def make_dirs():
    # make dirs for models & logs
    dirs = [
        "/content/models",

        "/content/models/segmentation",
        "/content/models/classification",
        "/content/models/autoencoder",

        "/content/models/segmentation/unet",
        "/content/models/segmentation/segformer",
        "/content/models/classification/efficienet",
        "/content/models/classification/convnext",

        "/content/models_log",

        "/content/models_log/segmentation",
        "/content/models_log/segmentation/unet",
        "/content/models_log/segmentation/segformer",

        "/content/models_log/classification",
        "/content/models_log/classification/efficienet",
        "/content/models_log/classification/convnext",
        "/content/models_log/autoencoder",
    ]
    for dir in dirs:
        os.makedirs(dir, exist_ok=True)



def mount_drive():

    drive.mount('/content/drive')
    source = '/content/drive/MyDrive/datasets/multi'  # Na Drive
    destination = '/content/datasets/multi'            # Link lokalny

    os.makedirs('/content/datasets', exist_ok=True)

    if os.path.exists(destination):
        if os.path.islink(destination):
            os.unlink(destination)
        else:
            shutil.rmtree(destination)

    # os.symlink(source, destination)
    print(f"Copying dataset from {source} to {destination}...")

    # Count files for progress bar
    total_files = sum([len(files) for r, d, files in os.walk(source)])

    with tqdm(total=total_files, desc="Copying", unit="file") as pbar:
        def copy_func(src, dst):
            shutil.copy2(src, dst)
            pbar.update(1)

        shutil.copytree(source, destination, copy_function=copy_func)

    print(f"Dataset copied!")
    print(f"  {destination} <- {source}")

def set_colab(path: str, on_colab: bool):
    dir = Path(path)
    result = list(dir.rglob("hparams.[pP][yY]"))
    for r in result:
        with open(r, "r+") as f:
            pos = f.tell()
            line = f.readline()
            if on_colab:
                if line == "ON_COLAB = False\n":
                    f.seek(pos)
                    f.write("ON_COLAB = True\n")
            else:
                if line == "ON_COLAB = True\n":
                    f.seek(pos)
                    f.write("ON_COLAB = False\n")



In [ ]:
make_dirs()

In [ ]:
mount_drive()

Mounted at /content/drive
Copying dataset from /content/drive/MyDrive/datasets/multi to /content/datasets/multi...


Copying: 100%|██████████| 26200/26200 [10:09<00:00, 42.98file/s] 

Dataset copied!
  /content/datasets/multi <- /content/drive/MyDrive/datasets/multi


In [ ]:
set_colab("/content/bachlor_backup/src", True)

In [ ]:

!cat bachlor_backup/src/segmentation/common/hparams.py | head -1
!cat bachlor_backup/src/classification/common/hparams.py | head -1
!cat bachlor_backup/src/autoencoder/hparams.py | head -1
!cat bachlor_backup/src/dim_translator/hparams.py | head -1


ON_COLAB = True
ON_COLAB = True
ON_COLAB = True
cat: bachlor_backup/src/dim_translator/hparams.py: No such file or directory


In [ ]:
!rm -rd bachlor_backup/

In [ ]:
!cd bachlor_backup/src/autoencoder && python train.py

In [ ]:

original_sys_path = sys.path.copy()

sys.path.append('/content/bachlor_backup/src/autoencoder')
import train as autoencoder_train

sys.path.append('/content/bachlor_backup/src/segmentation/unet/')
import train as unet_train

sys.path.append('/content/bachlor_backup/src/segmentation/segformer/')
import train as segformer_train

sys.path.append('/content/bachlor_backup/src/classification/convnext')
import train as convnext_train

sys.path.append('/content/bachlor_backup/src/classification/efficienet')
import train as efficienet_train
